In [2]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

urls = [
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S09.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S10.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S11.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S12.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S13.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S14.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S15.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S16.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S17.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S18.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S19.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S20.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U07.htm"
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U08.htm"
]

def extract_state_name(url):
    return url.split('-')[-1].split('.')[0]

def process_url(url):
    state_name = extract_state_name(url)  
    directory = f'./{state_name}'
    if not os.path.exists(directory):
        os.makedirs(directory)   
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    party_seats = []
    grid_boxes = soup.find_all("div", class_="grid-box")
    for box in grid_boxes:
        party_name = box.find("h4").text
        seats_won = box.find("h2").text
        party_seats.append({"Party": party_name, "Seats": int(seats_won)})
    
    script = soup.find("script", string=lambda text: text and "var xValues" in text).string

    x_values_pattern = re.compile(r"var xValues = \[(.*?)\];", re.DOTALL)
    y_values_pattern = re.compile(r"var yValues = \[(.*?)\];", re.DOTALL)

    x_values_match = x_values_pattern.search(script)
    y_values_match = y_values_pattern.search(script)

    x_values = x_values_match.group(1).split(',')
    y_values = y_values_match.group(1).split(',')

    x_values = [x.strip().strip("'").strip('"') for x in x_values]
    y_values = [y.strip() for y in y_values]

    x_values = [x for x in x_values if x]
    y_values = [int(y) for y in y_values if y.isdigit()]

    party_vote_share = []
    for i in range(len(x_values)):
        party_vote_share.append({"Party": x_values[i], "Votes": y_values[i]})
    df_seats = pd.DataFrame(party_seats)
    df_vote_share = pd.DataFrame(party_vote_share)
    seat_plot_path = os.path.join(directory, f'{state_name}_party_seat_distribution.png')
    vote_share_plot_path = os.path.join(directory, f'{state_name}_party_vote_share.png')
    df_seats.plot(kind="bar", x="Party", y="Seats", title=f"Party-wise Seat Distribution ({state_name})", legend=False)
    plt.ylabel("Seats Won")
    plt.savefig(seat_plot_path)
    plt.close()
    df_vote_share.plot(kind="pie", y="Votes", labels=df_vote_share["Party"], autopct='%1.1f%%', title=f"Party-wise Vote Share ({state_name})")
    plt.ylabel("")
    plt.savefig(vote_share_plot_path)
    plt.close()

    csv_file_path = os.path.join(directory, f'{state_name}_election_results.csv')
    with open(csv_file_path, 'w') as file:
        file.write(f"Party-wise Seat Distribution ({state_name}):\n")
        df_seats.to_csv(file, index=False)
        file.write("\nPlot: " + seat_plot_path + "\n\n")    
        file.write(f"Party-wise Vote Share ({state_name}):\n")
        df_vote_share.to_csv(file, index=False)
        file.write("\nPlot: " + vote_share_plot_path + "\n\n")
    print(f"Data and plots saved for {state_name}")

for url in urls:
    process_url(url)


Data and plots saved for S01
Data and plots saved for S02
Data and plots saved for S03
Data and plots saved for S04
Data and plots saved for S05
Data and plots saved for S06
Data and plots saved for S07
Data and plots saved for S08


AttributeError: 'NoneType' object has no attribute 'string'

In [5]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import base64
from io import BytesIO
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage
from openpyxl.styles import Font, Alignment
from openpyxl.utils import get_column_letter

urls = [
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S09.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S10.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S11.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S12.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S13.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S14.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S15.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S16.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S17.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S18.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S19.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S20.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U07.htm"
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U08.htm",
]


def extract_state_name(url):
    return url.split("-")[-1].split(".")[0]


def plot_to_base64(plt):
    buf = BytesIO()
    plt.savefig(buf, format="png", dpi=300, bbox_inches="tight")
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode("utf-8")


def create_dashboard(state_name, df_seats, df_vote_share, directory):
    seat_plot = df_seats.plot(
        kind="bar",
        x="Party",
        y="Seats",
        title=f"Party-wise Seat Distribution ({state_name})",
        legend=False,
        figsize=(10, 6),
    )
    plt.ylabel("Seats Won")
    seat_plot_img = plot_to_base64(plt)
    plt.close()

    vote_share_plot = df_vote_share.plot(
        kind="pie",
        y="Votes",
        labels=df_vote_share["Party"],
        autopct="%1.1f%%",
        title=f"Party-wise Vote Share ({state_name})",
        figsize=(10, 6),
    )
    plt.ylabel("")
    vote_share_plot_img = plot_to_base64(plt)
    plt.close()

    html_content = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Election Results Dashboard - {state_name}</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                color: #333;
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
                background-color: #f4f4f4;
            }}
            h1, h2 {{
                color: #2c3e50;
            }}
            .dashboard-container {{
                display: flex;
                flex-wrap: wrap;
                justify-content: space-between;
            }}
            .dashboard-item {{
                flex-basis: calc(50% - 20px);
                background-color: #fff;
                border-radius: 8px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
                margin-bottom: 20px;
                padding: 20px;
                box-sizing: border-box;
            }}
            table {{
                border-collapse: collapse;
                width: 100%;
                margin-bottom: 20px;
            }}
            th, td {{
                border: 1px solid #ddd;
                padding: 12px;
                text-align: left;
            }}
            th {{
                background-color: #f2f2f2;
                font-weight: bold;
            }}
            img {{
                max-width: 100%;
                height: auto;
                margin-bottom: 20px;
            }}
            .toggle-btn {{
                background-color: #3498db;
                color: #fff;
                border: none;
                padding: 10px 15px;
                cursor: pointer;
                border-radius: 4px;
                font-size: 16px;
            }}
            .toggle-btn:hover {{
                background-color: #2980b9;
            }}
            .hidden {{
                display: none;
            }}
        </style>
    </head>
    <body>
        <h1>Election Results Dashboard - {state_name}</h1>
        <div class="dashboard-container">
            <div class="dashboard-item">
                <h2>Party-wise Seat Distribution</h2>
                <button class="toggle-btn" onclick="toggleVisibility('seats-table', 'seats-plot')">Toggle Table/Plot</button>
                <div id="seats-table">
                    <table>
                        <tr><th>Party</th><th>Seats</th></tr>
                        {df_seats.to_html(index=False, header=False)}
                    </table>
                </div>
                <div id="seats-plot" class="hidden">
                    <img src="data:image/png;base64,{seat_plot_img}" alt="Seat Distribution Plot">
                </div>
            </div>
            <div class="dashboard-item">
                <h2>Party-wise Vote Share</h2>
                <button class="toggle-btn" onclick="toggleVisibility('vote-table', 'vote-plot')">Toggle Table/Plot</button>
                <div id="vote-table">
                    <table>
                        <tr><th>Party</th><th>Votes</th></tr>
                        {df_vote_share.to_html(index=False, header=False)}
                    </table>
                </div>
                <div id="vote-plot" class="hidden">
                    <img src="data:image/png;base64,{vote_share_plot_img}" alt="Vote Share Plot">
                </div>
            </div>
        </div>
        <script>
            function toggleVisibility(tableId, plotId) {{
                var table = document.getElementById(tableId);
                var plot = document.getElementById(plotId);
                if (table.classList.contains('hidden')) {{
                    table.classList.remove('hidden');
                    plot.classList.add('hidden');
                }} else {{
                    table.classList.add('hidden');
                    plot.classList.remove('hidden');
                }}
            }}
        </script>
    </body>
    </html>
    """

    dashboard_path = os.path.join(directory, f"{state_name}_dashboard.html")
    with open(dashboard_path, "w") as file:
        file.write(html_content)

    print(f"Dashboard created for {state_name}: {dashboard_path}")


def process_url(url):
    state_name = extract_state_name(url)
    directory = f"./{state_name}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    party_seats = []
    grid_boxes = soup.find_all("div", class_="grid-box")
    for box in grid_boxes:
        party_name = box.find("h4").text
        seats_won = box.find("h2").text
        party_seats.append({"Party": party_name, "Seats": int(seats_won)})

    script = soup.find(
        "script", string=lambda text: text and "var xValues" in text
    ).string

    x_values_pattern = re.compile(r"var xValues = \[(.*?)\];", re.DOTALL)
    y_values_pattern = re.compile(r"var yValues = \[(.*?)\];", re.DOTALL)

    x_values_match = x_values_pattern.search(script)
    y_values_match = y_values_pattern.search(script)

    x_values = x_values_match.group(1).split(",")
    y_values = y_values_match.group(1).split(",")

    x_values = [x.strip().strip("'").strip('"') for x in x_values]
    y_values = [y.strip() for y in y_values]

    x_values = [x for x in x_values if x]
    y_values = [int(y) for y in y_values if y.isdigit()]

    party_vote_share = []
    for i in range(len(x_values)):
        party_vote_share.append({"Party": x_values[i], "Votes": y_values[i]})

    df_seats = pd.DataFrame(party_seats)
    df_vote_share = pd.DataFrame(party_vote_share)

    seat_plot_path = os.path.join(
        directory, f"{state_name}_party_seat_distribution.png"
    )
    vote_share_plot_path = os.path.join(directory, f"{state_name}_party_vote_share.png")

    df_seats.plot(
        kind="bar",
        x="Party",
        y="Seats",
        title=f"Party-wise Seat Distribution ({state_name})",
        legend=False,
        figsize=(10, 6),
    )
    plt.ylabel("Seats Won")
    plt.savefig(seat_plot_path)
    plt.close()

    df_vote_share.plot(
        kind="pie",
        y="Votes",
        labels=df_vote_share["Party"],
        autopct="%1.1f%%",
        title=f"Party-wise Vote Share ({state_name})",
        figsize=(10, 6),
    )
    plt.ylabel("")
    plt.savefig(vote_share_plot_path)
    plt.close()

    create_dashboard(state_name, df_seats, df_vote_share, directory)

    excel_file_path = os.path.join(directory, f"{state_name}_election_results.xlsx")
    wb = Workbook()
    ws = wb.active
    ws.title = "Election Results"

    ws.cell(
        row=1, column=1, value=f"Party-wise Seat Distribution ({state_name})"
    ).font = Font(bold=True)
    for col, header in enumerate(["Party", "Seats"], start=1):
        ws.cell(row=2, column=col, value=header).font = Font(bold=True)
    for row, data in enumerate(df_seats.itertuples(index=False), start=3):
        for col, value in enumerate(data, start=1):
            ws.cell(row=row, column=col, value=value)
    start_row = len(df_seats) + 5
    ws.cell(
        row=start_row, column=1, value=f"Party-wise Vote Share ({state_name})"
    ).font = Font(bold=True)
    for col, header in enumerate(["Party", "Votes"], start=1):
        ws.cell(row=start_row + 1, column=col, value=header).font = Font(bold=True)
    for row, data in enumerate(
        df_vote_share.itertuples(index=False), start=start_row + 2
    ):
        for col, value in enumerate(data, start=1):
            ws.cell(row=row, column=col, value=value)
    img_seat = XLImage(seat_plot_path)
    img_vote = XLImage(vote_share_plot_path)

    ws.add_image(img_seat, "E2")
    ws.add_image(img_vote, f"E{start_row}")

    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = max_length + 2
        ws.column_dimensions[column].width = adjusted_width

    wb.save(excel_file_path)

    print(f"Data, plots, and Excel file saved for {state_name}")


for url in urls:
    process_url(url)

Dashboard created for S27: ./S27\S27_dashboard.html
Data, plots, and Excel file saved for S27
Dashboard created for S10: ./S10\S10_dashboard.html
Data, plots, and Excel file saved for S10
Dashboard created for S11: ./S11\S11_dashboard.html
Data, plots, and Excel file saved for S11
Dashboard created for U09: ./U09\U09_dashboard.html
Data, plots, and Excel file saved for U09
Dashboard created for U06: ./U06\U06_dashboard.html
Data, plots, and Excel file saved for U06
Dashboard created for S12: ./S12\S12_dashboard.html
Data, plots, and Excel file saved for S12
Dashboard created for S06: ./S06\S06_dashboard.html
Data, plots, and Excel file saved for S06
Dashboard created for S07: ./S07\S07_dashboard.html
Data, plots, and Excel file saved for S07
Dashboard created for S08: ./S08\S08_dashboard.html
Data, plots, and Excel file saved for S08
Dashboard created for U08: ./U08\U08_dashboard.html
Data, plots, and Excel file saved for U08


In [ ]:
<select id="ctl00_ContentPlaceHolder1_Result1_ddlState" onchange="return GetResult(this)" name="state">
	<option value=""> Select State Wise </option>
	<option value="U01">Andaman &amp; Nicobar Islands</option>
	<option value="S01">Andhra Pradesh</option>
	<option value="S02">Arunachal Pradesh</option>
	<option value="S03">Assam</option>
	<option value="S04">Bihar</option>
	<option value="U02">Chandigarh</option>
	<option value="S26">Chhattisgarh</option>
	<option value="U03">Dadra &amp; Nagar Haveli and Daman &amp; Diu</option>
	<option value="S05">Goa</option>
	<option value="S06">Gujarat</option>
	<option value="S07">Haryana</option>
	<option value="S08">Himachal Pradesh</option>
	<option value="U08">Jammu and Kashmir</option>
	<option value="S27">Jharkhand</option>
	<option value="S10">Karnataka</option>
	<option value="S11">Kerala</option>
	<option value="U09">Ladakh</option>
	<option value="U06">Lakshadweep</option>
	<option value="S12">Madhya Pradesh</option>
	<option value="S13">Maharashtra</option>
	<option value="S14">Manipur</option>
	<option value="S15">Meghalaya</option>
	<option value="S16">Mizoram</option>
	<option value="S17">Nagaland</option>
	<option value="U05">NCT OF Delhi</option>
	<option value="S18">Odisha</option>
	<option value="U07">Puducherry</option>
	<option value="S19">Punjab</option>
	<option value="S20">Rajasthan</option>
	<option value="S21">Sikkim</option>
	<option value="S22">Tamil Nadu</option>
	<option value="S29">Telangana</option>
	<option value="S23">Tripura</option>
	<option value="S24">Uttar Pradesh</option>
	<option value="S28">Uttarakhand</option>
	<option value="S25">West Bengal</option>
</select>